In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxPath=os.path.sep.join([r'deploy.prototxt'])
wghtsPath=os.path.sep.join([r'res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
prototxPath

'deploy.prototxt'

In [4]:
wghtsPath

'res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxPath,wghtsPath)

In [6]:
model=load_model(r'maskdetector.model')

In [7]:
#loading an image to test
img=cv2.imread(r'example4.png')

In [8]:
img

array([[[ 48,  54,  59],
        [ 48,  54,  59],
        [ 48,  54,  59],
        ...,
        [129, 128, 130],
        [129, 128, 130],
        [130, 129, 131]],

       [[ 48,  54,  59],
        [ 47,  53,  58],
        [ 48,  54,  59],
        ...,
        [131, 130, 132],
        [131, 130, 132],
        [131, 130, 132]],

       [[ 49,  55,  60],
        [ 49,  55,  60],
        [ 48,  54,  59],
        ...,
        [132, 131, 133],
        [133, 132, 134],
        [130, 129, 131]],

       ...,

       [[103, 110, 149],
        [104, 111, 150],
        [101, 108, 147],
        ...,
        [202, 204, 228],
        [202, 204, 228],
        [201, 203, 227]],

       [[106, 113, 152],
        [107, 114, 153],
        [ 99, 106, 145],
        ...,
        [202, 204, 228],
        [202, 204, 228],
        [202, 204, 228]],

       [[ 99, 108, 146],
        [104, 113, 151],
        [100, 109, 147],
        ...,
        [206, 208, 232],
        [206, 208, 232],
        [207, 208, 229]]

In [9]:
(h,w)=img.shape[:2]

In [10]:
(h,w)

(1051, 939)

In [11]:
blob=cv2.dnn.blobFromImage(img,1.0,(300,300),(104.0,177.0,123.0))

In [12]:
blob

array([[[[ -57.,  -54.,  -56., ...,   25.,   25.,   27.],
         [ -55.,  -54.,  -56., ...,   30.,   28.,   27.],
         [ -55.,  -54.,  -56., ...,   27.,   30.,   28.],
         ...,
         [   3.,   -1.,   -5., ...,   98.,   98.,   98.],
         [  -6.,    1.,    0., ...,   98.,   98.,   98.],
         [   2.,   -1.,    1., ...,   98.,   98.,   98.]],

        [[-124., -121., -123., ...,  -49.,  -49.,  -47.],
         [-122., -121., -123., ...,  -44.,  -46.,  -47.],
         [-122., -121., -123., ...,  -47.,  -44.,  -46.],
         ...,
         [ -61.,  -65.,  -68., ...,   27.,   27.,   27.],
         [ -72.,  -65.,  -66., ...,   27.,   27.,   27.],
         [ -64.,  -67.,  -65., ...,   27.,   27.,   27.]],

        [[ -65.,  -62.,  -64., ...,    7.,    7.,    9.],
         [ -63.,  -62.,  -64., ...,   12.,   10.,    9.],
         [ -63.,  -62.,  -64., ...,    9.,   12.,   10.],
         ...,
         [  31.,   27.,   24., ...,  105.,  105.,  105.],
         [  21.,   28.,   

In [13]:
blob.shape

(1, 3, 300, 300)

In [14]:
net.setInput(blob)
dtection=net.forward()

In [15]:
dtection

array([[[[0.        , 1.        , 0.99957174, ..., 0.10500476,
          0.71922636, 0.64488685],
         [0.        , 1.        , 0.1778277 , ..., 0.4191376 ,
          1.0715886 , 1.0622818 ],
         [0.        , 1.        , 0.11848845, ..., 4.0076714 ,
          4.839086  , 4.9851522 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [16]:
# detections loop
for i in range(0,dtection.shape[2]):
    confidence=dtection[0,0,i,2]
    
    if confidence>0.5:
        box=dtection[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        # bound boxes should fall within the dimensions 
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        #extract  ROI, resizing it to 224,224 and preprocess it
        face=img[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        #Prob in model
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        #display the label and bound box
        cv2.putText(img,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(img,(startX,startY),(endX,endY),color,2)
                
#display output         
cv2.imshow("OutPut",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 3s 3s/step
